In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

2022-11-15 16:48:16.853587: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 16:48:16.853608: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
conc = pd.read_csv('torchsaved/fr_enbleurtconcat.csv')

In [11]:
conc.iloc[39]

Unnamed: 0                                                   63
src           Les stigmatiser pour en faire des parias risqu...
hyp           (China must learn to behave “responsibly,” McC...
ref           (China must learn to behave “responsibly,” McC...
score                                                  0.954767
qbin                                                        NaN
Name: 39, dtype: object

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512").to("cuda:0")
model.eval()

In [2]:
# 8 example thing seems to mess up the dataset btw
testset = pd.read_csv("/mnt/data1/prasann/latticegen/lattice-generation/torchsaved/bleurtfr_entrain.csv")

In [3]:

import matplotlib.pyplot as plt

In [4]:
# algorithm that takes in distribution, returns widely spaced sample
def get_distrib_bins(inputs):
    

SyntaxError: unexpected EOF while parsing (4010794027.py, line 3)

In [ ]:
testset['score'][tind*32:(tind+1)*32]

In [6]:
# let's say we want to get 4 batches from 32 (including ref, which is gold)
def get_bin_sample(subdf):
    #print(subdf)
    bins = pd.qcut(subdf['score'], 4, precision=1)
    ref = subdf.iloc[-1]
    subdf['qbin'] = bins
    ubins = bins.unique()
    res = []
    for i in range(0, len(ubins)-1):
        tmp = subdf[subdf['qbin']==ubins[i]].sample(replace=True)
        res.append(tmp.iloc[-1])
    res.append(ref)
    return pd.DataFrame(res).drop(columns=['Unnamed: 0'])

def convert_bin_sample(fulldf):
    assert len(fulldf)%32==0
    
    print(len(fulldf)/32)
    newdf = pd.DataFrame()
    fullres = []
    for i in range(int(len(fulldf)/32)):
        for j in range(8):
            fullres.append(get_bin_sample(fulldf.iloc[i*32: (i+1)*32]))
        if i%500==0:
            print(i)
    return pd.concat(fullres, axis=0)
    

In [7]:
concatdata = convert_bin_sample(testset)
concatdata.to_csv('torchsaved/bleurtfr_enex8train.csv')

4666.0
0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [11]:
concatdata.iloc[32:40]

,src,hyp,ref,score,qbin
34,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.238013,"(-0.4, -0.2]"
47,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.146433,"(-0.2, -0.1]"
50,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.082636,"(-0.1, -0.04]"
63,Les stigmatiser pour en faire des parias risqu...,"(China must learn to behave “responsibly,” McC...","(China must learn to behave “responsibly,” McC...",0.954767,NaN
34,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.238013,"(-0.4, -0.2]"
45,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.150359,"(-0.2, -0.1]"
50,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.082636,"(-0.1, -0.04]"
63,Les stigmatiser pour en faire des parias risqu...,"(China must learn to behave “responsibly,” McC...","(China must learn to behave “responsibly,” McC...",0.954767,NaN


In [8]:
get_bin_sample(testset.iloc[0*32: (0+1)*32])

,src,hyp,ref,score,qbin
6,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" an editorial i...","""Just how tainted has clinical medicine become...",-0.045865,"(-0.30000000000000004, 0.01]"
12,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" asked an edito...","""Just how tainted has clinical medicine become...",0.035191,"(0.01, 0.08]"
22,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" an editorial i...","""Just how tainted has clinical medicine become...",0.218525,"(0.08, 0.2]"
31,« La médecine clinique est-elle corrompue ? » ...,"""Just how tainted has clinical medicine become...","""Just how tainted has clinical medicine become...",0.981153,NaN


In [134]:
concatdata.iloc[:8]

,Unnamed: 0,src,hyp,ref,score,qbin
2,5250,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" an editorial i...","""Just how tainted has clinical medicine become...",-0.103205,"(-0.4, -0.1]"
7,5255,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" asked an edito...","""Just how tainted has clinical medicine become...",-0.000347,"(-0.1, -0.0003]"
10,5258,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" asked an edito...","""Just how tainted has clinical medicine become...",0.027366,"(-0.0003, 0.03]"
13,5261,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" asked an edito...","""Just how tainted has clinical medicine become...",0.049029,"(0.03, 0.08]"
19,5267,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" an editorial i...","""Just how tainted has clinical medicine become...",0.132007,"(0.08, 0.1]"
22,5270,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" an editorial i...","""Just how tainted has clinical medicine become...",0.218525,"(0.1, 0.2]"
27,5275,« La médecine clinique est-elle corrompue ? » ...,"""Is clinical medicine corrupt?"" asked an edito...","""Just how tainted has clinical medicine become...",0.316283,"(0.2, 0.3]"
32,5312,Les stigmatiser pour en faire des parias risqu...,"In fact, Russia has almost cooperated with the...","(China must learn to behave “responsibly,” McC...",-0.289452,"(-0.4, -0.1]"


In [ ]:
tind = 21
pd.qcut(testset['score'][tind*32:(tind+1)*32], 8, precision=0)

In [ ]:
testset.iloc[31]['ref']

In [ ]:
references = ['"Just how tainted has clinical medicine become," asked an editorial in the British medical journal The Lancet in 2002; the answer was "heavily and damagingly so." ']
candidates = ['"Is clinical medicine corrupt?" an editorial in the British medical journal The Lancet asked in 2002, with only one answer: strongly, not harmlessly.']

with torch.no_grad():
    inputs = tokenizer(references, candidates, return_tensors='pt').to("cuda:0")
    scores = model(**inputs)[0].squeeze()

#print(list(scores)) # tensor([1.0327, 0.2055])

In [ ]:
scores